# All API calls to Assemble Dataframe

In [1]:
import pandas as pd 
import numpy as np
import datetime
from datetime import date
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import os
import io
import pickle

In [2]:
pd.set_option('display.max_columns', None)

## Functions

In [3]:
def dates_to_str(last_date = date.today(), historical_days = 1450):
    first_date = last_date-datetime.timedelta(days=historical_days)
    return first_date.strftime('%Y-%m-%d'), last_date.strftime('%Y-%m-%d')

In [4]:
def fetch_stock(symbol, last_date = date.today(), historical_days = 1450):
    '''
    Get the trading information about a stock for a range of days in "historical_days" before the "last_date"
    The output is a DataFrame with columns "close","high","low","open","volume","splitFactor"
    The output are adjusted prices
    '''
    first_date, curr_date = dates_to_str(last_date, historical_days)
    

    #The request itself
    url = f'https://api.tiingo.com/tiingo/daily/{symbol}/prices?startDate={first_date}&endDate={curr_date} '
    headers = {
            'Content-Type': 'application/json',
            'Authorization' : f'Token {Tiingo_API}'
            }
    r = requests.get(url, headers=headers)
    response = r.json()
    response = pd.DataFrame(response, index=np.arange(0,len(response)))
    response.set_index(["date"], inplace = True)
    response.index = pd.to_datetime(response.index)
    response.drop(columns = ["close","high","low","open","volume","splitFactor"],axis = 1, inplace = True)
    response.rename(columns = {"adjClose":"close","adjHigh":"high","adjLow":"low","adjOpen":"open","adjVolume":"volume"}, inplace = True)
    return response

In [5]:
def fetch_fundamentals(symbol, last_date = date.today(), historical_days = 1450):
    '''
    The output is a dataframe with daily fundamentals:
    Market_Cap, Enterprise_Value, PE_Ratio, PB_Ratio, Trailing PEG
    '''
    #Get latest and historical day, month, year for API request
    first_date, curr_date = dates_to_str(last_date, historical_days)
    
    url = f'https://api.tiingo.com/tiingo/fundamentals/{symbol}/daily?token={Tiingo_API}?startDate={first_date}&endDate={curr_date}'
    headers = {
            'Content-Type': 'application/json',
            'Authorization' : f'Token {Tiingo_API}'
            }
    r = requests.get(url, headers=headers)
    response = r.json()
    response = pd.DataFrame(response, index=np.arange(0,len(response)))
    response.set_index(["date"], inplace = True)
    response.index = pd.to_datetime(response.index)
    return response

In [6]:
def fetch_statements(symbol, last_date = date.today(), historical_days = 1450):
    '''
    Gets historical financial data about the stock like Total Assets, Accounts Payable, Short & Long term Debts etc.
    '''
    #Get latest and historical day, month, year for API request
    hist_date_str, latest_date_str = dates_to_str(last_date, historical_days)
    
    url = f'https://api.tiingo.com/tiingo/fundamentals/{symbol}/statements?token={Tiingo_API}?startDate={hist_date_str}&endDate={latest_date_str}'
    headers = {
            'Content-Type': 'application/json',
            'Authorization' : f'Token {Tiingo_API}'
            }
    r = requests.get(url, headers=headers)
    response = r.json()
    final_df = pd.DataFrame()
    
    
    for i in range(len(response)):
        balanceSheet_df = pd.DataFrame(response[i]["statementData"]["balanceSheet"]).T
        overview_df = pd.DataFrame(response[i]["statementData"]["overview"]).T
        cashFlow_df = pd.DataFrame(response[i]["statementData"]["cashFlow"]).T
        incomeStatement_df = pd.DataFrame(response[i]["statementData"]["incomeStatement"]).T
        dfs = [balanceSheet_df, overview_df, cashFlow_df, incomeStatement_df]
        general_df = pd.concat(dfs, axis =1)
        
        #Make 1st row a header
        new_header = list(general_df.loc["dataCode"]) #grab the first row for the header
        general_df.drop("dataCode",axis = 0, inplace = True) #take the data less the header row
        general_df.columns = new_header
        
        general_df["date"] = response[i]["date"]
        general_df["quarter"] = response[i]["quarter"]
        general_df["year"] = response[i]["year"]
        print(f'Combining Statements for {response[i]["year"]} and {response[i]["quarter"]}')
        final_df = pd.concat([final_df, general_df], axis = 0)
    final_df.set_index(["date"], inplace = True)
    #converting date from str to datetime object
    final_df.index = pd.to_datetime(final_df.index, yearfirst = True, utc = True, origin = "unix")
    final_df_copy = final_df.copy()
    #Dealing with duplicated indexes of datetime
    if final_df.index.duplicated().sum()>0:
        duplicates_in_df = final_df.index.duplicated()
        final_df.dropna(axis = 0, inplace = True)
    final_df = final_df.apply(pd.to_numeric, errors='ignore')
    return final_df

In [7]:
def fetch_metadata(): 
    '''
    Get's metadata about all available stocks, like:
        Industry, Location, Full Name, Ticker, etc.
    '''
    url = f'https://api.tiingo.com/tiingo/fundamentals/meta?token={Tiingo_API}'
    headers = {
            'Content-Type': 'application/json',
            'Authorization' : f'Token {Tiingo_API}'
            }
    r = requests.get(url, headers=headers)
    response = r.json()
    response = pd.DataFrame(response)
    return response

In [8]:
def combine_tables(stock_df, statemets_data, fundamentals_data, dates_range, todays_date = date.today()):
    
    '''
    Combining into one Dataframe information about daily trading values, statements, fundamentals
    '''
    #We create an index of dates range
    dates = pd.date_range(todays_date-datetime.timedelta(days=dates_range),todays_date,freq='d')    
    #We change the name to make join with other dataframes available by the same index
    dates.rename('date', inplace = True)
    #Convert index to dataframe
    dates_df = pd.DataFrame(index = dates)
    #Delete timezone from the statements to make join available
    statemets_data.index = statemets_data.index.tz_convert(None)
    stock_df.index = stock_df.index.tz_convert(None)
    fundamentals_data.index = fundamentals_data.index.tz_convert(None)
    
    dates_and_statemts = dates_df.merge(statemets_data, on='date', how='left')
    #Statements table contains data about quarters only while dates DF is much bigger. We populate quarter results to days
    dates_and_statemts.ffill(axis = 0, inplace = True)
    stock_df = stock_df.merge(dates_and_statemts, on='date', how = 'left')
    stock_df = stock_df.merge(fundamentals_data, on='date', how = 'left')
    return stock_df

In [9]:
def search_symbol(symbol, driver):
    elem = driver.find_element(By.ID,"ticker")
    elem.send_keys(symbol)
    elem.send_keys(Keys.RETURN)
    driver.implicitly_wait(3)

In [10]:
def find_estim_tables(driver):
    '''
    get a table with earnings on a page  ZACK
    '''
    #Find required table
    elem = driver.find_element(By.XPATH,'//*[@id="earnings_announcements_earnings_table"]') 
    #get all rows from the table
    rows = [row.text.encode("utf8") for row in elem.find_elements(By.TAG_NAME,'tr')]
    #convert list of bytes to list of strings
    rows = [row.decode("utf-8").replace("\n"," ") for row in rows]
    #convert list of strings to a dataframe 
    earnings_history_df = pd.read_csv(io.StringIO('\n'.join(rows)), delim_whitespace=True, header = 0, names = ['date', 'Period_Ending', 'Estimate', 'Reported', 'Surprise', 'Surprise_%', "str1", "str2"])
    
    #Find the latest estimate 
    elem = driver.find_element(By.XPATH,'//*[@id="right_content"]/section[2]/div') 
    #get all rows from the table
    rows1 = [row.text.encode("utf8") for row in elem.find_elements(By.TAG_NAME,'td')]
    rows2 = [row.text.encode("utf8") for row in elem.find_elements(By.TAG_NAME,'th')]
    #convert list of bytes to list of strings
    rows1 = [row.decode("utf-8").replace("\n"," ") for row in rows1]
    rows2 = [row.decode("utf-8").replace("\n"," ") for row in rows2]    
    #convert list of strings to a dataframe 
    earnings_latest_df = pd.DataFrame(rows1).transpose()
    earnings_latest_df.columns =  ['Period_Ending', 'Estimate', 'Surprise_%']
    earnings_latest_df["date"] = rows2[-1].split(" ")[0]
    
    earnings = pd.concat([earnings_latest_df, earnings_history_df], ignore_index = True, sort = False)
    earnings['date'] = pd.to_datetime(earnings['date'])
    earnings.set_index(["date"], inplace = True)
    
    return earnings

In [11]:
def find_divid_tables(driver):
    '''
    get a table with dividends on a page  ZACK
    '''
    element = driver.find_element(By.XPATH,'//*[@id="earnings_announcements_tabs"]/ul')
    driver.execute_script('arguments[0].scrollIntoView({block: "center", inline: "center"})', element)
    
    elem = driver.find_element(By.XPATH,'//*[@id="ui-id-7"]')
    elem.click()
    time.sleep(3)
    #Expand 100 records
    dropdown = driver.find_element(By.NAME,"earnings_announcements_dividends_table_length")
    Select(dropdown).select_by_visible_text("100")
    
    #Find required table
    elem = driver.find_element(By.XPATH,'//*[@id="earnings_announcements_dividends_table"]') 
    #get all rows from the table
    rows = [row.text.encode("utf8") for row in elem.find_elements(By.TAG_NAME,'tr')]
    #convert list of bytes to list of strings
    rows = [row.decode("utf-8").replace("\n"," ") for row in rows]
    #convert list of strings to a dataframe 
    
    dividends_history_df = pd.read_csv(io.StringIO('\n'.join(rows)), delim_whitespace=True, header = 0, names = ['Date_Paid', 'Amount', 'Date_Announced', 'Ex-Dividend_Date'])
    dividends_history_df.dropna(axis = 0, inplace = True)
    dividends_history_df['Date_Paid'] = pd.to_datetime(dividends_history_df['Date_Paid'])
    dividends_history_df['Date_Announced'] = pd.to_datetime(dividends_history_df['Date_Announced'])
    dividends_history_df['Ex-Dividend_Date'] = pd.to_datetime(dividends_history_df['Ex-Dividend_Date'])
    dividends_history_df.rename(columns = {"Date_Announced":"date"}, inplace = True)
    dividends_history_df.set_index(["date"], inplace = True)
    
    return dividends_history_df

In [12]:
def get_earn_and_dividends(symbol):
    '''
    This function launches browser for data load and fetches earnings and dividends data
    '''
    #Start Chrome 
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    #Go to the website
    driver.get('https://www.zacks.com/stock/research/CSCO/earnings-calendar')
    
    #Search stock
    search_symbol(symbol, driver)
    
    #expand_100_earnings values:
    dropdown = driver.find_element(By.NAME,"earnings_announcements_earnings_table_length")
    Select(dropdown).select_by_visible_text("100")
    time.sleep(3)
    
    #Get earnings
    earnings = find_estim_tables(driver)
    time.sleep(3)
    
    #Ge dividends
    dividends = find_divid_tables(driver)
    
    #Close browser
    driver.close()
    
    #Transforming Earnings dataframe to a final version
    #Transform string values to numeric
    earnings.replace({"--":np.nan},inplace = True)
    earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
    earnings["surprise_%"] = earnings["Surprise_%"]/100
    earnings["date_of_report"] = earnings.index
    #getting expected future earnings change
    earnings["future_estimate"] = earnings.Estimate.shift(1)
    earnings["previous_surprise"] = earnings["surprise_%"].shift(-1)
    earnings["expected_growth"]= (earnings.future_estimate - earnings.Reported)/earnings.Reported
    earnings = earnings[["surprise_%", "expected_growth", "previous_surprise", "date_of_report"]]
    
    #Transforming Dividends dataframe to a final version
    #STR to value
    dividends.replace({"--":np.nan},inplace = True)
    dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))
    #get date that we later can use to count days after the announcement
    dividends["date_announced"] = dividends.index
    #Getting dividends trend
    dividends["previous_divid"] = dividends.Amount.shift(-1)
    dividends["dividends_change"] = (dividends.Amount - dividends.previous_divid)/dividends.previous_divid
    dividends = dividends[dividends.dividends_change != 0]
    dividends["prev_div_change"] = dividends.dividends_change.shift(-1)
    dividends = dividends[["dividends_change","prev_div_change","date_announced"]]
    
    #earnings = a.copy()
    #dividends = b.copy()
    
    #Match earnings with dates
    #Creating Dates dataframe with all possible dates values
    dates_df=pd.DataFrame()
    dates_df["date"] = pd.date_range(start=earnings.index.min(), end=earnings.index.max())
    #Set dates column as index
    dates_df.set_index(["date"], inplace = True)
    #Creating a dates_earnings dataset where we extrapolate existing quarterly data to daily
    dates_earnings = dates_df.copy()
    dates_earnings = dates_earnings.join(earnings, how = 'left')
    dates_earnings.sort_values(by = 'date', axis = 0, ascending = True, inplace = True)
    dates_earnings.ffill(axis = 0, inplace = True)
    dates_earnings.sort_values(by = 'date', axis = 0, ascending = False, inplace = True)
    dates_earnings["days_after_earn_report"] = dates_earnings.index - dates_earnings["date_of_report"] 
    dates_earnings['days_after_earn_report'] = pd.to_numeric(dates_earnings['days_after_earn_report'].dt.days, downcast='integer')
    dates_earnings.drop(["date_of_report"], axis = 1, inplace = True)
    
    #Match dividends with dates
    #Creating Dates dataframe with all possible dates values
    if dividends.empty:
        dates_dividends = pd.DataFrame(columns = ["days_after_divid_report", "dividends_change","prev_div_change"])
        dates_dividends.index.names = ['date']
    else:
        dates_df=pd.DataFrame()
        dates_df["date"] = pd.date_range(start=dividends.index.min(), end=date.today())
        #Set dates column as index
        dates_df.set_index(["date"], inplace = True)
        #Creating a dates_ividends dataset where we extrapolate existing quarterly data to daily
        dates_dividends = dates_df.copy()
        dates_dividends = dates_dividends.join(dividends, how = 'left')
        dates_dividends.sort_values(by = 'date', axis = 0, ascending = True, inplace = True)
        dates_dividends.ffill(axis = 0, inplace = True)
        dates_dividends.sort_values(by = 'date', axis = 0, ascending = False, inplace = True)
        dates_dividends["days_after_divid_report"] = dates_dividends.index - dates_dividends["date_announced"] 
        dates_dividends['days_after_divid_report'] = pd.to_numeric(dates_dividends['days_after_divid_report'].dt.days, downcast='integer')
        dates_dividends.drop(["date_announced"], axis = 1, inplace = True)
    
    return dates_earnings, dates_dividends

In [13]:
def jump_returns(dataset):
    dataset['15d_Return'] = -(dataset['close'] - dataset['close'].shift(-15)) / dataset['close']
    dataset['30d_Return'] = -(dataset['close'] - dataset['close'].shift(-30)) / dataset['close']
    dataset['60d_Return'] = -(dataset['close'] - dataset['close'].shift(-60)) / dataset['close']
    return dataset

# Assemble Complete Dataset

### Setting a few overall variables

In [100]:
#Enter TIINGO
print("Enter 40 signs tiingo API: ")  
Tiingo_API = input()
todays_date = date.today()
first = todays_date 
historical_dates_range = 1825
# ab72115a4a65c514653d132f6cddd6e1c3e0205d

Enter 40 signs tiingo API: 
ab72115a4a65c514653d132f6cddd6e1c3e0205d


In [101]:
# for add on
first = date.today()
historical_dates_range = (first - date(2023, 2, 27)).days

Of the ~25 input predictors, 3 are market condition features, which are scraped from the web only once, and not for each stock. The code blocks to pull these and write them to csv's are directly below, whereas the rest of the features are stock-specific, and are pulled after the list of functions.

## Scrape 10Y Bond Prices (Currently bugged, download manually)

In [ ]:
last_date = date.today() 
historical_days = 1825
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://finance.yahoo.com/quote/%5ETNX/history?period1=1492732800&period2=1650326400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true')

#remove old file with data if it's available
filePath = 'C:\\Users\\HP\\OneDrive\\Documents\\Self\\Projects\\Fundamental_Analysis\\TNX.csv'

# Check whether the  specified file is available and remove it  
if os.path.exists(filePath):
    os.remove(filePath)
    print("We remove old csv file with metadata and replace with the newer one")

#Calculating current and historical periods
todays_year = last_date.year
todays_month = last_date.month
todays_day = last_date.day
historical_date = last_date-datetime.timedelta(days=historical_days)
historical_year = historical_date.year
historical_month = historical_date.month
historical_day = historical_date.day

#Searching an input field for dates
elem = driver.find_element(By.XPATH,'//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/div/div/div/span')
elem.click()
time.sleep(10)

#Enter historical date
elem = driver.find_element(By.XPATH,'//*[@id="dropdown-menu"]/div/div[1]/input')
elem.send_keys(historical_year)
elem.send_keys(Keys.TAB)
elem.send_keys(historical_month)
elem.send_keys(historical_day)
time.sleep(8)

#Enter last date
elem = driver.find_element(By.XPATH,'//*[@id="dropdown-menu"]/div/div[2]/input')
elem.send_keys(todays_year)
elem.send_keys(Keys.TAB)
elem.send_keys(todays_month)
elem.send_keys(todays_day)
time.sleep(8)

#Press done after entering dates
elem = driver.find_element(By.XPATH,'//*[@id="dropdown-menu"]/div/div[3]/button[1]')
elem.click()
time.sleep(5)

#Press apply dates range
elem = driver.find_element(By.XPATH,'//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/button')
elem.click()
time.sleep(5)

#press download data
elem = driver.find_element(By.XPATH,'//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[2]/span[2]/a/span')
elem.click()
time.sleep(3)

driver.close

TNX = pd.read_csv("C:\\Users\\HP\\OneDrive\\Documents\\Self\\Projects\\Fundamental_Analysis\\TNX.csv")

TNX.Date = pd.to_datetime(data.Date)
TNX.rename(columns = {"Date":"date", "Adj Close":"10Y_bonds"}, inplace = True)
TNX.set_index(["date"], inplace = True)
TNX = data["10Y_bonds"]
TNX.dropna(axis = 0, inplace = True)



In [81]:
TNX = pd.read_csv("C:\\Users\\HP\\OneDrive\\Documents\\Self\\Projects\\Fundamental_Analysis\\TNX.csv")

In [113]:
# Due to 30d bond return feature, put addon as YTD instead of simply days since last recorded
TNX_addon = pd.read_csv("C:\\Users\\HP\\OneDrive\\Documents\\Self\\Projects\\Fundamental_Analysis\\TNX_addon.csv")

## Download volatilty index

In [114]:
def get_vix(last_date = date.today(), historical_days = 1450):
    historical_date = last_date-datetime.timedelta(days=historical_days)
    
    url="https://cdn.cboe.com/api/global/us_indices/daily_prices/VIX_History.csv"
    response=pd.read_csv(url)
    
    response["DATE"]=pd.to_datetime(response["DATE"])
    response.rename(columns = {"DATE":"Date"}, inplace = True)
    response.set_index(["Date"], inplace = True)
    response = response[historical_date:last_date]["HIGH"]
    response = response.rename("VIX_high")
    
    return response

## Import S&P500 List and construct loop to pull data

In [115]:
TNX = TNX.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])
TNX = TNX.loc[~TNX['Close'].isna()]

KeyError: "['Open', 'High', 'Low', 'Adj Close', 'Volume'] not found in axis"

In [116]:
# for add on
TNX_addon = TNX_addon.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])
TNX_addon = TNX_addon.loc[~TNX_addon['Close'].isna()]

In [134]:
df_seed = TNX_addon
#df_seed = TNX
df_seed['Date'] = pd.to_datetime(df_seed['Date'])
df_seed = df_seed.set_index('Date')
df_seed['30dBondReturn'] = (df_seed['Close'] - df_seed['Close'].shift(30)) / df_seed['Close'].shift(30)
df_seed.columns = ['10YBond', '30dBondRet']
df_seed = df_seed.loc[~df_seed['30dBondRet'].isna()]

In [140]:
df_seed

,10YBond,30dBondRet,VIX_high
Date,,,
2023-02-28,3.916,0.115352,21.37


In [136]:
# for addon
df_seed = df_seed.loc[df_seed.index > pd.to_datetime('2023-02-27')]

In [137]:
vix = pd.DataFrame(get_vix(date.today(), historical_dates_range))

In [138]:
df_seed = df_seed.merge(vix, on='Date', how='left')
#df_seed = df_seed.set_index('Date')

In [139]:
# tickers = pd.read_csv('S&P500_List.txt')
tickers = pd.read_csv('DOW30_List.csv')

#### Get SPY S&P price

In [141]:
spy = fetch_stock('SPY', date.today())
spy = spy.drop(columns=['high', 'low', 'open', 'volume', 'divCash'])
spy.rename(columns = {"close":"spy_close"}, inplace = True)
spy.index = spy.index.tz_convert(None)
spy = spy.reset_index()

In [142]:
spy_mapping = spy['spy_close'].to_dict()

In [143]:
spy['spy_5dret'] = spy.index - 5
spy['spy_5dret'] = (spy.index.map(spy_mapping) - spy['spy_5dret'].map(spy_mapping)) / spy['spy_5dret'].map(spy_mapping)
spy['spy_10dret'] = spy.index - 10 
spy['spy_10dret'] = (spy.index.map(spy_mapping) - spy['spy_10dret'].map(spy_mapping)) / spy['spy_10dret'].map(spy_mapping)
spy['spy_15dret'] = spy.index  - 15 
spy['spy_15dret'] = (spy.index.map(spy_mapping) - spy['spy_15dret'].map(spy_mapping)) / spy['spy_15dret'].map(spy_mapping)


In [144]:
df_seed = df_seed.merge(spy, left_on='Date', right_on='date', how='left')

In [92]:
df_seed = df_seed[df_seed['date']>pd.to_datetime('2019-01-01')]

In [145]:
df_seed

,10YBond,30dBondRet,VIX_high,date,spy_close,spy_5dret,spy_10dret,spy_15dret
0,3.916,0.115352,21.37,2023-02-28,396.26,-0.007091,-0.040138,-0.033111


#### Activate cell immediately below for creating a new df

In [96]:
df_total = pd.DataFrame()

#### Activate cell immediately below for tacking on to existing df saved to folder

In [147]:
df_total = pd.read_csv('All_Data.csv')
df_total = df_total.set_index('date')

### Loop over stocks in list

In [98]:
# tickers.iloc[i,0] is ticker, tickers.iloc[i,2] is sector
# first is 2/17/23
# historical_dates_range = 120

for i in range(16,len(tickers)):
    # base_df is VIX and TNX info that goes along with each stock
    base_df = df_seed.copy()
    # scrape earnings and dividends from the web
    earnings, dividends = get_earn_and_dividends(tickers.iloc[i,0])
    # Fetch stock data, fundamentals, and fundamentals from Tiingo api
    
    stock_dataset = fetch_stock(tickers.iloc[i,0], first, historical_dates_range)
    fundamentals = fetch_fundamentals(tickers.iloc[i,0], first, historical_dates_range)
    
    statements = fetch_statements(tickers.iloc[i,0], first, historical_dates_range)
    
    
    # Combining all stock's data
    big_dataset = combine_tables(stock_dataset, statements, fundamentals, historical_dates_range)

    big_dataset.sort_values(by = 'date', axis = 0, ascending = False, inplace = True)
    big_dataset["Debt-to-Equity_Ratio"] = big_dataset["totalAssets"]/big_dataset["totalLiabilities"]
    big_dataset["DividendsYield"] = big_dataset["payDiv"]/big_dataset["marketCap"]
    big_dataset["PayoutRatio"] = big_dataset["payDiv"]/big_dataset["grossProfit"]
    big_dataset["Acc_Rec_Pay_Ration"] = big_dataset["acctRec"]/big_dataset["acctPay"]
    big_dataset["Earnings_per_stock"] = big_dataset["epsDil"]/big_dataset["close"]
    
    
    # forming complete df for one individual stock
    
    base_df = base_df.reset_index().merge(earnings, left_on='date', right_on='date', how='left').set_index('date')
    base_df = base_df.reset_index().merge(dividends, left_on='date', right_on='date', how='left').set_index('date')
    base_df = base_df.reset_index().merge(big_dataset, left_on='date', right_on='date', how='left').set_index('date')
    base_df = jump_returns(base_df)
    base_df['Stock'] = tickers.iloc[i,0]
    # sample only one data point per week
    
    # base_df = base_df.iloc[len(base_df)-725:len(base_df),:]
    #base_df = base_df.reset_index()
    #base_df = base_df[base_df.index % 7 == 1]
    #base_df = base_df.set_index('Date')
    
    
    # concatenate entire stock's results to overall df_total
    
    df_total = pd.concat([df_total,base_df])
    print(f'Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i={i+1}')

C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))


Combining Statements for 2023 and 3
Combining Statements for 2023 and 2
Combining Statements for 2023 and 1
Combining Statements for 2022 and 4
Combining Statements for 2022 and 0
Combining Statements for 2022 and 3
Combining Statements for 2022 and 2
Combining Statements for 2022 and 1
Combining Statements for 2021 and 4
Combining Statements for 2021 and 0
Combining Statements for 2021 and 3
Combining Statements for 2021 and 2
Combining Statements for 2021 and 1
Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i=17


C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))


Combining Statements for 2022 and 4
Combining Statements for 2022 and 0
Combining Statements for 2022 and 3
Combining Statements for 2022 and 2
Combining Statements for 2022 and 1
Combining Statements for 2021 and 4
Combining Statements for 2021 and 0
Combining Statements for 2021 and 3
Combining Statements for 2021 and 2
Combining Statements for 2021 and 1
Combining Statements for 2020 and 4
Combining Statements for 2020 and 0
Combining Statements for 2020 and 3
Combining Statements for 2020 and 2
Combining Statements for 2020 and 1
Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i=18


C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))


...

Combining Statements for 2022 and 4
Combining Statements for 2022 and 0
Combining Statements for 2022 and 3
Combining Statements for 2022 and 2
Combining Statements for 2022 and 1
Combining Statements for 2021 and 4
Combining Statements for 2021 and 0
Combining Statements for 2021 and 3
Combining Statements for 2021 and 2
Combining Statements for 2021 and 1
Combining Statements for 2020 and 4
Combining Statements for 2020 and 0
Combining Statements for 2020 and 3
Combining Statements for 2020 and 2
Combining Statements for 2020 and 1
Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i=28


C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))


Combining Statements for 2023 and 1
Combining Statements for 2022 and 4
Combining Statements for 2022 and 0
Combining Statements for 2022 and 3
Combining Statements for 2022 and 2
Combining Statements for 2022 and 1
Combining Statements for 2021 and 4
Combining Statements for 2021 and 0
Combining Statements for 2021 and 3
Combining Statements for 2021 and 2
Combining Statements for 2021 and 1
Combining Statements for 2020 and 4
Combining Statements for 2020 and 0
Combining Statements for 2020 and 3
Combining Statements for 2020 and 2
Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i=29


C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))


Combining Statements for 2022 and 4
Combining Statements for 2022 and 0
Combining Statements for 2022 and 3
Combining Statements for 2022 and 2
Combining Statements for 2022 and 1
Combining Statements for 2021 and 4
Combining Statements for 2021 and 0
Combining Statements for 2021 and 3
Combining Statements for 2021 and 2
Combining Statements for 2021 and 1
Combining Statements for 2020 and 4
Combining Statements for 2020 and 0
Combining Statements for 2020 and 3
Combining Statements for 2020 and 2
Combining Statements for 2020 and 1
Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i=30


## Loop to add additional data on top of df_total ending at 2/17

In [31]:
df_total = pd.read_csv('All_Data.csv')
df_total = df_total.set_index('date') 

In [148]:
first = date.today()
historical_dates_range = (first - date(2023, 2, 27)).days

In [151]:
for i in range(13,len(tickers)):
    # base_df is VIX and TNX info that goes along with each stock
    base_df = df_seed.copy()
    # scrape earnings and dividends from the web
    earnings, dividends = get_earn_and_dividends(tickers.iloc[i,0])
    # Fetch stock data, fundamentals, and fundamentals from Tiingo api
    
    stock_dataset = fetch_stock(tickers.iloc[i,0], first, historical_dates_range)
    fundamentals = fetch_fundamentals(tickers.iloc[i,0], first, historical_dates_range)
    
    statements = fetch_statements(tickers.iloc[i,0], first, historical_dates_range)
    
    
    # Combining all stock's data
    big_dataset = combine_tables(stock_dataset, statements, fundamentals, historical_dates_range)

    big_dataset.sort_values(by = 'date', axis = 0, ascending = False, inplace = True)
    big_dataset["Debt-to-Equity_Ratio"] = big_dataset["totalAssets"]/big_dataset["totalLiabilities"]
    big_dataset["DividendsYield"] = big_dataset["payDiv"]/big_dataset["marketCap"]
    big_dataset["PayoutRatio"] = big_dataset["payDiv"]/big_dataset["grossProfit"]
    big_dataset["Acc_Rec_Pay_Ration"] = big_dataset["acctRec"]/big_dataset["acctPay"]
    big_dataset["Earnings_per_stock"] = big_dataset["epsDil"]/big_dataset["close"]
    
    
    # forming complete df for one individual stock
    
    base_df = base_df.reset_index().merge(earnings, left_on='date', right_on='date', how='left').set_index('date')
    base_df = base_df.reset_index().merge(dividends, left_on='date', right_on='date', how='left').set_index('date')
    base_df = base_df.reset_index().merge(big_dataset, left_on='date', right_on='date', how='left').set_index('date')
    base_df = jump_returns(base_df)
    base_df['Stock'] = tickers.iloc[i,0]
    # sample only one data point per week
    
    # base_df = base_df.iloc[len(base_df)-725:len(base_df),:]
    #base_df = base_df.reset_index()
    #base_df = base_df[base_df.index % 7 == 1]
    #base_df = base_df.set_index('Date')
    
    
    # concatenate entire stock's results to overall df_total
    
    df_total = pd.concat([df_total,base_df])
    print(f'Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i={i+1}')

C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))


Combining Statements for 2022 and 4
Combining Statements for 2022 and 0
Combining Statements for 2022 and 3
Combining Statements for 2022 and 2
Combining Statements for 2022 and 1
Combining Statements for 2021 and 4
Combining Statements for 2021 and 0
Combining Statements for 2021 and 3
Combining Statements for 2021 and 2
Combining Statements for 2021 and 1
Combining Statements for 2020 and 4
Combining Statements for 2020 and 0
Combining Statements for 2020 and 3
Combining Statements for 2020 and 2
Combining Statements for 2020 and 1
Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i=14


C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))


Combining Statements for 2023 and 4
Combining Statements for 2023 and 0
Combining Statements for 2023 and 3
Combining Statements for 2023 and 2
Combining Statements for 2023 and 1
Combining Statements for 2022 and 4
Combining Statements for 2022 and 0
Combining Statements for 2022 and 3
Combining Statements for 2022 and 2
Combining Statements for 2022 and 1
Combining Statements for 2021 and 0
Combining Statements for 2020 and 4
Combining Statements for 2020 and 3
Combining Statements for 2020 and 2
Combining Statements for 2020 and 1
Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i=15


C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))


...

Combining Statements for 2022 and 4
Combining Statements for 2022 and 0
Combining Statements for 2022 and 3
Combining Statements for 2022 and 2
Combining Statements for 2022 and 1
Combining Statements for 2021 and 4
Combining Statements for 2021 and 0
Combining Statements for 2021 and 3
Combining Statements for 2021 and 2
Combining Statements for 2021 and 1
Combining Statements for 2020 and 4
Combining Statements for 2020 and 0
Combining Statements for 2020 and 3
Combining Statements for 2020 and 2
Combining Statements for 2020 and 1
Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i=28


C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))


Combining Statements for 2023 and 1
Combining Statements for 2022 and 4
Combining Statements for 2022 and 0
Combining Statements for 2022 and 3
Combining Statements for 2022 and 2
Combining Statements for 2022 and 1
Combining Statements for 2021 and 4
Combining Statements for 2021 and 0
Combining Statements for 2021 and 3
Combining Statements for 2021 and 2
Combining Statements for 2021 and 1
Combining Statements for 2020 and 4
Combining Statements for 2020 and 0
Combining Statements for 2020 and 3
Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i=29


C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
C:\Users\HP\AppData\Local\Temp\ipykernel_17716\3181593439.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))


Combining Statements for 2022 and 4
Combining Statements for 2022 and 0
Combining Statements for 2022 and 3
Combining Statements for 2022 and 2
Combining Statements for 2022 and 1
Combining Statements for 2021 and 4
Combining Statements for 2021 and 0
Combining Statements for 2021 and 3
Combining Statements for 2021 and 2
Combining Statements for 2021 and 1
Combining Statements for 2020 and 4
Combining Statements for 2020 and 0
Combining Statements for 2020 and 3
Combining Statements for 2020 and 2
Combining Statements for 2020 and 1
Loop Interrupt: If this is the last Loop Interrupt message, restart loop at i=30


In [79]:
statements

,debtCurrent,taxAssets,investmentsCurrent,totalAssets,acctPay,accoci,inventory,totalLiabilities,acctRec,intangibles,ppeq,deferredRev,cashAndEq,assetsNonCurrent,taxLiabilities,investments,equity,retainedEarnings,deposits,assetsCurrent,investmentsNonCurrent,debt,debtNonCurrent,liabilitiesNonCurrent,liabilitiesCurrent,sharesBasic,longTermDebtEquity,shareFactor,bookVal,roa,currentRatio,roe,grossMargin,piotroskiFScore,epsQoQ,revenueQoQ,profitMargin,rps,bvps,ncfi,capex,ncfx,ncff,sbcomp,ncf,payDiv,businessAcqDisposals,issrepayDebt,issrepayEquity,investmentsAcqDisposals,freeCashFlow,ncfo,depamor,ebitda,netIncComStock,epsDil,consolidatedIncome,nonControllingInterests,shareswaDil,intexp,rnd,eps,netIncDiscOps,grossProfit,shareswa,opex,ebt,netinc,revenue,ebit,taxExp,opinc,sga,costRev,prefDVDs,quarter,year
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-12-31,4.367000e+09,0.0,1.719400e+10,1.821030e+11,9.595000e+09,-5.620000e+08,1.322400e+10,7.881700e+10,4.133000e+09,3.360900e+10,8.086000e+10,0.000000e+00,1.114400e+10,1.316960e+11,6.249000e+09,2.310600e+10,1.014230e+11,7.040500e+10,0.0,5.040700e+10,5.912000e+09,4.205100e+10,3.768400e+10,4.666200e+10,3.215500e+10,4.127000e+09,0.371553,1.0,1.032860e+11,0.045552,1.567626,0.079021,0.391682,3.0,-1.141593,-0.315959,0.391682,3.402472,25.026896,-3.431000e+09,-5.699000e+09,0.0,2.343000e+09,736000000.0,6.615000e+09,-1.509000e+09,0.000000e+00,2.490000e+09,5.000000e+06,1.194000e+09,2.004000e+09,7.703000e+09,3.287000e+09,2.338000e+09,-6.640000e+08,-0.16,-6.610000e+08,3000000.0,4.123000e+09,-150000000.0,4.464000e+09,-0.16,0.0,5.500000e+09,4.120000e+09,6.632000e+09,-7.990000e+08,-6.640000e+08,1.404200e+10,-9.490000e+08,-1.350000e+08,-1.132000e+09,1.706000e+09,8.542000e+09,0.0,4,2022
2022-10-01,2.283000e+09,0.0,1.803000e+10,1.748410e+11,7.133000e+09,-2.051000e+09,1.283100e+10,7.495600e+10,7.469000e+09,3.385900e+10,7.576300e+10,0.000000e+00,4.529000e+09,1.255780e+11,4.143000e+09,2.385200e+10,9.988500e+10,7.102400e+10,0.0,4.926300e+10,5.822000e+09,3.952300e+10,3.724000e+10,4.714300e+10,2.781300e+10,4.106000e+09,0.372829,1.0,9.988500e+10,0.077105,1.771222,0.133133,0.426066,3.0,-0.851190,-0.200813,0.426066,3.735509,24.326595,-4.574000e+09,-7.299000e+09,0.0,3.683000e+09,793000000.0,1.390000e+08,-1.502000e+09,0.000000e+00,4.661000e+09,3.830000e+08,3.672000e+09,-6.269000e+09,1.030000e+09,3.252000e+09,2.926000e+09,1.019000e+09,0.25,1.019000e+09,0.0,4.125000e+09,-138000000.0,4.302000e+09,0.25,0.0,6.535000e+09,4.118000e+09,6.710000e+09,-1.880000e+08,1.019000e+09,1.533800e+10,-3.260000e+08,-1.207000e+09,-1.750000e+08,1.744000e+09,8.803000e+09,0.0,3,2022
2022-07-02,2.882000e+09,0.0,2.265400e+10,1.704180e+11,7.945000e+09,-1.625000e+09,1.217400e+10,6.920000e+10,6.063000e+09,3.401400e+10,7.166000e+10,0.000000e+00,4.390000e+09,1.198300e+11,4.256000e+09,2.858300e+10,1.012180e+11,7.298500e+10,0.0,5.058800e+10,5.929000e+09,3.543000e+10,3.254800e+10,4.198200e+10,2.721800e+10,4.089000e+09,0.321563,1.0,1.012180e+11,0.111865,1.858623,0.196033,0.364663,4.0,-1.088000,-0.219551,0.364663,3.746882,24.753730,1.680000e+08,-7.255000e+09,0.0,-2.802000e+09,892000000.0,-1.825000e+09,-1.499000e+09,3.500000e+07,-1.688000e+09,0.000000e+00,8.587000e+09,-6.446000e+09,8.090000e+08,3.148000e+09,2.358000e+09,-4.540000e+08,-0.11,-4.540000e+08,0.0,4.100000e+09,119000000.0,4.400000e+09,-0.11,0.0,5.587000e+09,4.100000e+09,6.287000e+09,-9.090000e+08,-4.540000e+08,1.532100e+10,-7.900000e+08,-4.550000e+08,-7.000000e+08,1.800000e+09,9.734000e+09,0.0,2,2022
2022-04-02,4.459000e+09,0.0,3.248100e+10,1.763560e+11,7.210000e+09,-1.002000e+09,1.193500e+10,7.322000e+10,7.074000e+09,3.382400e+10,6.671800e+10,0.000000e+00,6.215000e+09,1.137880e+11,5.919000e+09,3.851700e+10,1.031360e+11,7.489400e+10,0.0,6.256800e+10,6.036000e+09,3.724700e+10,3.278800e+10,4.389800e+10,2.932200e+10,4.072000e+09,0.317910,1.0,1.031360e+11,0.147575,2.133824,0.263442,0.503678,5.0,1.397590,-0.067097,0.503678,4.507122,25.328094,-2.640000e+09,-4.797000e

### Log of completed iterations

In [ ]:
# DOW30 Complete in All_Data.csv, saved to df_DOW_30.csv
# DOW30 Recent data (2023) in Recent_DOW30.csv
# DOW30 entirely raw data saved to All_Data.csv (no weekly sampling, lag and future return calcs fixed, NaN's managed)

In [152]:
df_total.to_csv('All_Data.csv', header = df_total.columns, index = True, encoding = 'utf-8')